In [67]:
#Packages
import os
import pandas as pd
import itertools
import numpy as np
from operator import itemgetter

In [68]:
#Load Data
moving_average_shares = pd.read_csv('../temporary/moving_average_shares.csv', index_col='Unnamed: 0')
topics = pd.read_csv('../temporary/topics.csv')
moving_average_shares

,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,...,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890
1x2,0.000010,1.993856e-06,1.890401e-06,1.797580e-06,1.797580e-06,0.000025,0.000023,0.000023,0.000023,0.000023,...,0.002293,0.002321,0.002335,0.002333,0.002247,0.002226,0.002238,0.002245,0.002236,0.002221
1x3,0.000005,1.068301e-06,1.005507e-06,9.554093e-07,9.554093e-07,0.000012,0.000011,0.000011,0.000011,0.000011,...,0.000669,0.000691,0.000721,0.000737,0.000755,0.000758,0.000769,0.000787,0.000802,0.000786
1x4,0.000009,2.563470e-06,2.364779e-06,2.242189e-06,2.242189e-06,0.000020,0.000019,0.000019,0.000019,0.000019,...,0.000735,0.000739,0.000704,0.000693,0.000674,0.000659,0.000636,0.000626,0.000647,0.000630
1x5,0.000014,1.480231e-06,1.488265e-06,1.423555e-06,1.423555e-06,0.000212,0.000199,0.000199,0.000199,0.000199,...,0.001772,0.001853,0.001881,0.001907,0.001958,0.001996,0.002015,0.002068,0.002129,0.002141
1x6,0.000002,1.352189e-07,1.484902e-07,1.431995e-07,1.431995e-07,0.000005,0.000006,0.000006,0.000006,0.000006,...,0.000304,0.000328,0.000336,0.000359,0.000374,0.000373,0.000368,0.000363,0.000350,0.000349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57x59,0.000011,1.248851e-05,1.004193e-04,9.509673e-05,9.509673e-05,0.000084,0.000079,0.000079,0.000079,0.000079,...,0.000035,0.000035,0.000033,0.000033,0.000033,0.000033,0.000032,0.000033,0.000032,0.000033
57x60,0.000002,1.767700e-07,7.619845e-04,7.197240e-04,7.197240e-04,0.000632,0.000595,0.000595,0.000595,0.000595,...,0.000128,0.000133,0.000133,0.000142,0.000153,0.000153,0.000149,0.000148,0.000153,0.000155
58x59,0.003315,3.998958e-03,3.537325e-03,3.338580e-03,3.338580e-03,0.002930,0.002758,0.002758,0.002758,0.002758,...,0.000198,0.000202,0.000204,0.000199,0.000200,0.000204,0.000205,0.000209,0.000214,0.000214
58x60,0.000104,1.116548e-04,9.957842e-05,9.400082e-05,9.400082e-05,0.000086,0.000081,0.000081,0.000081,0.000081,...,0.000916,0.000920,0.000944,0.000950,0.000983,0.001013,0.001030,0.001046,0.001066,0.001057


In [69]:
topics

,1,2,topic_number
0,0.14322,paint pictur artist music engrav painter colou...,1
1,0.25957,town road church build built river stone wall ...,2
2,0.12400,franc pari french loui madam duke count napole...,3
3,0.19923,church christian christ bishop holi paul doctr...,4
4,0.36643,love heart beauti soul sweet dark night earth ...,5
5,0.05915,india chines china nativ indian bengal govern ...,6
6,0.15907,fig water iron engin pressur steam electr air ...,7
7,0.07679,acid solut heat carbon water sulphur iron gas ...,8
8,0.49381,exist refer period similar consist occur conne...,9
9,0.07330,vol lond fol folio calf copi pari par morocco ...,10


In [77]:
def category_shares(topics, ctshares, year, categories):
    #'categories' needs to be a dict with keys as category names and values as a list of category topics, i.e. 'Category': [1,2,3]
    #'ctshares' needs to be a dataframe with cross_topic shares as values, years as columns, and indices as 'topic1 x topic2', e.g. '3x4'
    #topics must have a column 'topic_number' with the number corresponding to each topic
    tmp_dict = {}
    
    topic_numbers = list(topics['topic_number'])
    shares = ctshares[str(year)] #grab column with cross-topic shares for the year

    for name,category in categories.items():
        combos = {topic:list(itertools.product([topic], set(category) - set([topic]))) for topic in topic_numbers} #This gets the appropriate combo of each topic and the topics in each category, so for topic 1 and 'Political Economy', gets [(1,33),(1,34),(1,47)]

        cross_combos = {key:['x'.join(map(str,sorted(i))) for i in value] for key,value in combos.items()} #sorts topic pairs so that they will be called from 'shares' correctly, and joins with 'x'

        cross_shares = {key:[shares[str(i)] for i in value] for key,value in cross_combos.items()} #get share value from 'shares'

        cross_sum = {key:sum(value) if key not in category else sum(value)*1.5 for key,value in cross_shares.items()} #sum shares for the topic and topics in category. If topic in category, multiply by 1.5 since it will be missing one topic pair, e.g. for topic 33 it just has 33x34 and 33x47, so must be scaled correctly. NOT ROBUST YET TO OTHER SIZES THAN 3 TOPICS PER CATEGORY.

        tmp_dict[name] = cross_sum

    df = pd.DataFrame.from_dict(tmp_dict)

    total = df.sum(axis=1) #get total of category scores
    cat_shares = df.div(total, axis=0) #divide by total, so that Religion + Science + Political Economy = 1
    
    return cat_shares

In [78]:
categories = {
    'Religion':[4,12,52],
    'Science':[7,8,41],
    'Political Economy':[33,34,47]
    }

years=[]
for year in range(1510,1891):
    years.append(year)


In [79]:
topic_shares = {}

for year in years:
    topic_shares[year] = category_shares(topics = topics, ctshares = moving_average_shares, year = year, categories = categories)

topic_shares[1550]

,Religion,Science,Political Economy
1,0.769908,0.001331,0.228760
2,0.799548,0.006658,0.193793
3,0.808955,0.002621,0.188424
4,0.809775,0.001638,0.188587
5,0.932808,0.000494,0.066698
6,0.928216,0.014389,0.057395
7,0.909221,0.006309,0.084470
8,0.932187,0.003544,0.064269
9,0.868286,0.005064,0.126650
10,0.802223,0.010861,0.186916


## Playground

In [26]:
category = [33,34,47]
shares = moving_average_shares['1550']
topic_numbers = list(topics['topic_number'])
# sets = {i:set([i] + category) for i in topic_numbers}
####USE SET(CATEGORY) - SET(TOPIC) TO DEAL WITH TOPICS IN CATEGORY
combos = {topic:list(itertools.product([topic], set(category) - set([topic]))) for topic in topic_numbers}
cross_combos = {key:['x'.join(map(str,sorted(i))) for i in value] for key,value in combos.items()}
cross_shares = {key:[shares[str(i)] for i in value] for key,value in cross_combos.items()}
cross_sum = {key:sum(value) if key not in category else sum(value)*1.5 for key,value in cross_shares.items()}

cross_shares

{1: [5.3437705817390094e-06, 2.1928275673091906e-07, 2.208617488631765e-06],
 2: [8.828332517273e-07, 4.6271290963295774e-08, 3.6239984159314877e-07],
 3: [1.7713311035523154e-07, 3.5106521150291194e-08, 6.216054912793866e-05],
 4: [5.98734347488627e-05, 2.5398373343049195e-06, 3.826067360469309e-05],
 5: [0.0001283015816262, 1.382151993499511e-07, 2.492053167066988e-07],
 6: [7.436954041344207e-08, 9.847696606732692e-09, 2.4659379558787974e-08],
 7: [6.480729282291057e-07, 1.7592576680011442e-08, 1.596683163102395e-07],
 8: [5.672311497556636e-07, 1.9302524022909407e-08, 6.879202922651646e-08],
 9: [1.6088664464288623e-06, 6.509095180336058e-08, 4.701732717830964e-07],
 10: [7.365102106311773e-06, 4.5466004034610614e-07, 6.578507286200873e-06],
 11: [1.6481838859184542e-06, 1.0336095562586363e-06, 2.114690148410176e-06],
 12: [0.001551711093789, 8.205941117116656e-06, 7.068716342686714e-05],
 13: [5.622943424208507e-07, 1.2703646157268686e-08, 4.2069174823597015e-08],
 14: [0.00011754

In [34]:
categories = {
    'Religion':[4,12,52],
    'Science':[7,8,41],
    'Political Economy':[33,34,47]
    }

for key,value in categories.items():
    print(key)
    print(value)

Religion
[4, 12, 52]
Science
[7, 8, 41]
Political Economy
[33, 34, 47]


In [30]:
moving_average_shares['1550']

1x2      6.046093e-08
1x3      2.046476e-07
1x4      4.012441e-06
1x5      4.252168e-07
1x6      7.824422e-09
             ...     
57x59    3.202851e-06
57x60    7.079243e-06
58x59    4.086480e-05
58x60    3.346370e-07
59x60    4.859878e-06
Name: 1550, Length: 1770, dtype: float64

In [18]:
s = {3,2,1}
print(sorted(s))

[1, 2, 3]


In [15]:
num = [1]

list(itertools.product(num, religion))

[(1, 2), (1, 3), (1, 4)]

In [ ]:
t =list(itemgetter(*ls)(moving_average_shares[1510]))
sum(t)

In [4]:
l1 = [1,2,3]
l2 = [4,5,6]
l3 = [7,8,9]

test = [l1,l2,l3]
for t in test:
    print(t)

[1, 2, 3]
[4, 5, 6]
[7, 8, 9]


In [11]:
mydict = {
    'one':l1,
    'two':l2,
    'three':l3}

for l in mydict:
    print(mydict[l])

df = pd.DataFrame.from_dict(mydict)
df

[1, 2, 3]
[4, 5, 6]
[7, 8, 9]


,one,two,three
0,1,4,7
1,2,5,8
2,3,6,9


In [16]:
moving_average_shares['1550']

1x2      6.046093e-08
1x3      2.046476e-07
1x4      4.012441e-06
1x5      4.252168e-07
1x6      7.824422e-09
             ...     
57x59    3.202851e-06
57x60    7.079243e-06
58x59    4.086480e-05
58x60    3.346370e-07
59x60    4.859878e-06
Name: 1550, Length: 1770, dtype: float64

In [13]:
topic = 2
cat = {2,3,4}
test = print(list(itertools.product([topic],cat - set([topic]))))
# cat - topic

[(2, 3), (2, 4)]


## Old Code

In [58]:
ct1 = []
ct2 = []
for i in range(1,61):
    for j in range(i+1,61):
        ct1.append(i)
        ct2.append(j)
moving_average_shares['Topic 1'] = ct1
moving_average_shares['Topic 2'] = ct2
moving_average_shares

,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,...,1883,1884,1885,1886,1887,1888,1889,1890,Topic 1,Topic 2
1x2,0.000010,1.993856e-06,1.890401e-06,1.797580e-06,1.797580e-06,0.000025,0.000023,0.000023,0.000023,0.000023,...,0.002335,0.002333,0.002247,0.002226,0.002238,0.002245,0.002236,0.002221,1,2
1x3,0.000005,1.068301e-06,1.005507e-06,9.554093e-07,9.554093e-07,0.000012,0.000011,0.000011,0.000011,0.000011,...,0.000721,0.000737,0.000755,0.000758,0.000769,0.000787,0.000802,0.000786,1,3
1x4,0.000009,2.563470e-06,2.364779e-06,2.242189e-06,2.242189e-06,0.000020,0.000019,0.000019,0.000019,0.000019,...,0.000704,0.000693,0.000674,0.000659,0.000636,0.000626,0.000647,0.000630,1,4
1x5,0.000014,1.480231e-06,1.488265e-06,1.423555e-06,1.423555e-06,0.000212,0.000199,0.000199,0.000199,0.000199,...,0.001881,0.001907,0.001958,0.001996,0.002015,0.002068,0.002129,0.002141,1,5
1x6,0.000002,1.352189e-07,1.484902e-07,1.431995e-07,1.431995e-07,0.000005,0.000006,0.000006,0.000006,0.000006,...,0.000336,0.000359,0.000374,0.000373,0.000368,0.000363,0.000350,0.000349,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57x59,0.000011,1.248851e-05,1.004193e-04,9.509673e-05,9.509673e-05,0.000084,0.000079,0.000079,0.000079,0.000079,...,0.000033,0.000033,0.000033,0.000033,0.000032,0.000033,0.000032,0.000033,57,59
57x60,0.000002,1.767700e-07,7.619845e-04,7.197240e-04,7.197240e-04,0.000632,0.000595,0.000595,0.000595,0.000595,...,0.000133,0.000142,0.000153,0.000153,0.000149,0.000148,0.000153,0.000155,57,60
58x59,0.003315,3.998958e-03,3.537325e-03,3.338580e-03,3.338580e-03,0.002930,0.002758,0.002758,0.002758,0.002758,...,0.000204,0.000199,0.000200,0.000204,0.000205,0.000209,0.000214,0.000214,58,59
58x60,0.000104,1.116548e-04,9.957842e-05,9.400082e-05,9.400082e-05,0.000086,0.000081,0.000081,0.000081,0.000081,...,0.000944,0.000950,0.000983,0.001013,0.001030,0.001046,0.001066,0.001057,58,60


In [59]:
from collections import defaultdict

years=[]
for year in range(1510,1891):
    years.append(year)

ctopics3 = defaultdict(dict)
for ind,row in moving_average_shares.iterrows():
    t1 = row['Topic 1']
    t2 = row['Topic 2']
    key = str(int(t1)) + ',' + str(int(t2))
    for year in years:
        ctopics3[str(year)][key] = row[str(year)]

In [60]:
volume_topics = '../input/20191007_topics.txt'

topic_words = '../input/20191007_keys.txt'
topics = pd.read_csv(topic_words, sep = '\t', lineterminator = '\n', header=None)
# topics.drop(columns = 0, inplace = True)
topics.columns = ['topic_num','weight','words'] 
topics['topic_num'] = topics['topic_num'] + 1

words=[]
for i in range(0,60):
    words.append(topics.iloc[i,2].split(' ')) #convert string to list
    if (i==14):
        words[i][2] = str('differ')#ligature "ff" causes labeling problems in matplotlib
        words[i][4]= str('sufficient')
        words[i][5]= str('suffer')
        words[i] = ' '.join(words[i][:7])#slice first 9 words and convert back to string for wrapping
        
    else:
        words[i] = ' '.join(words[i][:7])#slice first 9 words and convert back to string for wrapping


In [61]:
cluster_rel = [4,12,52]
cluster_sci = [7,8,41]
cluster_pol = [33,34,47]

In [62]:
moving_totals = {}
for i in range(0,381):
    shares1 = []
    shares2 = []
    shares3 = []
    total_shares = []
    
    for j in range(1,61):
        c1 = 0.0
        c2 = 0.0
        c3 = 0.0
        if j not in cluster_rel:
            for k in range(0,3):
                if j < cluster_rel[k]:
                    a = str(int(j)) + ',' + str(int(cluster_rel[k]))
                elif j > cluster_rel[k]:
                    a = str(int(cluster_rel[k])) + ',' + str(int(j))
                if k == 0:
                    c1 = c1 + ctopics3[str(years[i])][a]
                else:
                    c1 = c1 + ctopics3[str(years[i])][a]
            shares1.append(c1)
        else:
            for k in range(0,3):
                if j < cluster_rel[k]:
                    a = str(int(j)) + ',' + str(int(cluster_rel[k]))
                elif j > cluster_rel[k]:
                    a = str(int(cluster_rel[k])) + ',' + str(int(j))
                elif j == cluster_rel[k]:
                    pass
                if k == 0:
                    c1 = c1 + ctopics3[str(years[i])][a]
                else:
                    c1 = c1 + ctopics3[str(years[i])][a]
            c1 = c1*1.5
            shares1.append(c1)
            
        if j not in cluster_sci:
            for k in range(0,3):
                if j < cluster_sci[k]:
                    b = str(int(j)) + ',' + str(int(cluster_sci[k]))
                elif j > cluster_sci[k]:
                    b = str(int(cluster_sci[k])) + ',' + str(int(j))
                if k == 0:
                    c2 = c2 + ctopics3[str(years[i])][b]
                else:
                    c2 = c2 + ctopics3[str(years[i])][b]
            shares2.append(c2)
        else:
            for k in range(0,3):
                if j < cluster_sci[k]:
                    b = str(int(j)) + ',' + str(int(cluster_sci[k]))
                elif j > cluster_sci[k]:
                    b = str(int(cluster_sci[k])) + ',' + str(int(j))
                elif j == cluster_sci[k]:
                    pass
                if k == 0:
                    c2 = c2 + ctopics3[str(years[i])][b]
                else:
                    c2 = c2 + ctopics3[str(years[i])][b]
            c2 = c2*1.5
            shares2.append(c2)
            
        if j not in cluster_pol:
            for k in range(0,3):
                if j < cluster_pol[k]:
                    c = str(int(j)) + ',' + str(int(cluster_pol[k]))
                elif j > cluster_pol[k]:
                    c = str(int(cluster_pol[k])) + ',' + str(int(j))
                if k == 0:
                    c3 = c3 + ctopics3[str(years[i])][c]
                else:
                    c3 = c3 + ctopics3[str(years[i])][c]
            shares3.append(c3)
        else:
            for k in range(0,3):
                if j < cluster_pol[k]:
                    c = str(int(j)) + ',' + str(int(cluster_pol[k]))
                elif j > cluster_pol[k]:
                    c = str(int(cluster_pol[k])) + ',' + str(int(j))
                elif j == cluster_pol[k]:
                    pass
                if k == 0:
                    c3 = c3 + ctopics3[str(years[i])][c]
                else:
                    c3 = c3 + ctopics3[str(years[i])][c]
            c3 = c3*1.5
            shares3.append(c3)

            
        total = c1 + c2 + c3
        total_shares.append(total)
    u = zip(shares1, shares2, shares3, total_shares)
    moving_totals[str(years[i])] = pd.DataFrame(u, columns=['Religion', 'Science', 'Political Economy', 'Total'])
    label = []
    for m in range (1,61):
        line = str(m) + ' - ' + words[m-1]
        label.append(line)

    num = []
    for m in range(1,61):
        num.append(m)

    color = []
    for m in range (1,61):
        if (m in cluster_rel) or (m in cluster_pol) or (m in cluster_sci):
            color.append('1')
        else:
            color.append('2')

    moving_totals[str(years[i])]['Topic'] = label
    moving_totals[str(years[i])]['Topic Num'] = num
# #     #Uses topics shares over whole corpus to calculate whether a topic is "science, religion, or politics"
#     moving_totals[str(years[i])]['Color'] = totals[['Religion','Science','Political Economy']].idxmax(axis=1)

# Uses 1850 as a basis for whether topic is "science, religion, or politics"
for i in range(0,381):
    moving_totals[str(years[i])]['Color'] = moving_totals['1850'][['Religion','Science','Political Economy']].idxmax(axis=1)


In [63]:
moving_totals['1850']

,Religion,Science,Political Economy,Total,Topic,Topic Num,Color
0,0.000930,0.001294,0.001633,0.003856,1 - paint pictur artist music engrav painter c...,1,Political Economy
1,0.001487,0.002568,0.003652,0.007708,2 - town road church build built river stone,2,Political Economy
2,0.000570,0.000138,0.001817,0.002526,3 - franc pari french loui madam duke count,3,Political Economy
3,0.016786,0.000155,0.007851,0.024792,4 - church christian christ bishop holi paul d...,4,Religion
4,0.010754,0.000617,0.004816,0.016187,5 - love heart beauti soul sweet dark night,5,Religion
5,0.000553,0.000185,0.002653,0.003391,6 - india chines china nativ indian bengal govern,6,Political Economy
6,0.000202,0.009900,0.003570,0.013672,7 - fig water iron engin pressur steam electr,7,Science
7,0.000078,0.008844,0.001085,0.010007,8 - acid solut heat carbon water sulphur iron,8,Science
8,0.007941,0.009906,0.017758,0.035605,9 - exist refer period similar consist occur c...,9,Political Economy
9,0.000883,0.000213,0.000355,0.001450,10 - vol lond fol folio calf copi pari,10,Religion
